In [ ]:
import json
import base64
import os
import textwrap
from openai import OpenAI

GPT4V_KEY = "your_api_key"

client = OpenAI(api_key=GPT4V_KEY)

def send_jpg_image_to_gpt4v(image_path, question, api_key, call_count):
    # read file
    with open(image_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('ascii')

    # send request
    result = client.chat.completions.create(
        model="gpt-4-1106-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question},
                    {
                        "type": "image",
                        "image": encoded_image
                    },
                ],
            }
        ],
        max_tokens=300,
    )

    call_count += 1

    print(f"API call count: {call_count}")

    result_text = textwrap.fill(result.choices[0].message.content, width=70)
    print(result_text)
    return result_text, call_count

call_count = 0

with open('/content/drive/MyDrive/experiment/Foggy_qa_caption.json', 'r', encoding='utf-8') as file:
    with open('/content/drive/MyDrive/experiment/Foggy_qa_caption_answers4.json', 'a', encoding='utf-8') as output_file:
        # read line
        for line in file:

            item = json.loads(line)
            image_name = item['image']
            question = item['question']
            image_path = os.path.join('/content/drive/MyDrive/Foggy Cityscapes Dataset/Foggy_150', image_name)
            text, call_count = send_jpg_image_to_gpt4v(image_path, question, GPT4V_KEY, call_count)

            if text:
                data1 = {
                    "question_id": item.get('question_id', ''),
                    "image": image_name,
                    "prompt": question,
                    "text": text
                }
                json.dump(data1, output_file)
                output_file.write('\n')


print(f"Total number of API calls made: {call_count}")